In [1]:
import random, gc, os, pickle, csv, time, re

import datasets.utils
import models.utils
from models.cls_oml_ori_v2 import OML
from models.base_models_ori import LabelAwareReplayMemory

import numpy as np

import higher
import torch
import torch.nn.functional as F
from torch.utils import data

# Constants

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')


BASE_MODEL_PATH = "/data/model_runs/original_oml/aOML-order1-2023-03-15-no12"
find_path = "id4"

model_name = [x for x in os.listdir(BASE_MODEL_PATH) if  re.search(f"{find_path}.*\.pt$", x)][0]
model_path = os.path.join(BASE_MODEL_PATH, model_name)
memory_name = [x for x in os.listdir(BASE_MODEL_PATH) if  re.search(f"{find_path}.*memory\.pickle$", x)][0]
memory_path = os.path.join(BASE_MODEL_PATH, memory_name)
print(f"Using model from {model_path}")
print(f"Using memory_path from {memory_path}")

use_db_cache = True
cache_dir = 'tmp'

Using model from /data/model_runs/original_oml/aOML-order1-2023-03-15-no12/OML-order1-id4-2023-03-15_14-27-32.837449.pt
Using memory_path from /data/model_runs/original_oml/aOML-order1-2023-03-15-no12/OML-order1-id4-2023-03-15_14-27-32.837483_memory.pickle


In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 0.001*5,
    "meta_lr": 3e-5,
    "model": "roberta",
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5,
    "write_prob": 0.1,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 8000,
    "task_aware": True,
    "reverse_support": True,
    "curriculum_replay": True,
    "pln": "1fc"
}
sort_score = True
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# Load Dataset

In [5]:
print('Loading the datasets')
test_datasets = []
for dataset_id in dataset_order_mapping[order]:
    test_dataset_file = os.path.join(cache_dir, f"{dataset_id}.cache")
    if os.path.exists(test_dataset_file):
        with open(test_dataset_file, 'rb') as f:
            test_dataset = pickle.load(f)
    else:
        test_dataset = datasets.utils.get_dataset_test("", dataset_id)
        print('Loaded {}'.format(test_dataset.__class__.__name__))
        test_dataset = datasets.utils.offset_labels(test_dataset)
        pickle.dump(test_dataset, open( test_dataset_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Pickle saved at {test_dataset_file}")
    test_datasets.append(test_dataset)
print('Finished loading all the datasets')

Loading the datasets
Finished loading all the datasets


# Load Model

In [6]:
learner = OML(device=device, n_classes=n_classes, **args)
print('Using {} as learner'.format(learner.__class__.__name__))
learner.load_model(model_path)
with open(memory_path, 'rb') as f:
#     learner.memory = pickle.load(f)
    memory_buffer = pickle.load(f)


2023-03-15 15:42:33,386 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
2023-03-15 15:42:33,389 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt from cache at /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
2023-03-15 15:42:34,661 - transformers.configuration_utils - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c2

Using OML as learner


In [7]:
# Setting up task dict for task-aware
memory_buffer.task_dict = {
    0: list(range(5, 9)), # AG
    1: list(range(0, 5)), # Amazon
    2: list(range(0, 5)), # Yelp
    3: list(range(9, 23)), # DBPedia
    4: list(range(23, 33)), # Yahoo
}

In [8]:
dataclass_mapper = {
    "AGNewsDataset": 0,
    "AmazonDataset": 1,
    "YelpDataset": 2,
    "DBPediaDataset": 3,
    "YahooAnswersDataset": 4
}
dataclass_mapper["AGNewsDataset"]

0

# Testing

Select specific column index per row
https://stackoverflow.com/questions/23435782/numpy-selecting-specific-column-index-per-row-by-using-a-list-of-indexes

In [9]:
def evaluate(dataloader, updates, mini_batch_size, dataname=""):
    learner.rln.eval()
    learner.pln.train()
    
    all_losses, all_predictions, all_labels, all_label_conf = [], [], [], []
    all_adaptation_time = []
    # Get Query set first. and then find supporting support set
    for query_idx, (query_text, query_labels) in enumerate(dataloader):
        print(f"Query ID {query_idx}/{len(dataloader)}")
        # The task id to optimize to for support set
        # task_idx = get_task_from_label_list(query_labels, memory_buffer.task_dict)
        task_idx = dataclass_mapper[dataname]
        
    
        support_set = []
        for _ in range(updates):
            text, labels = memory_buffer.read_batch_task(batch_size=mini_batch_size, task_idx=task_idx, sort_score=sort_score)
            support_set.append((text, labels))

        with higher.innerloop_ctx(learner.pln, learner.inner_optimizer,
                                  copy_initial_weights=False, track_higher_grads=False) as (fpln, diffopt):
            
            INNER_tic = time.time()
            # Inner loop
            task_predictions, task_labels = [], []
            support_loss = []
            for text, labels in support_set:
                labels = torch.tensor(labels).to(device)
                input_dict = learner.rln.encode_text(text)
                _repr = learner.rln(input_dict)
                output = fpln(_repr)
                loss = learner.loss_fn(output, labels)
                diffopt.step(loss)
                pred = models.utils.make_prediction(output.detach())
                support_loss.append(loss.item())
                task_predictions.extend(pred.tolist())
                task_labels.extend(labels.tolist())
            INNER_toc = time.time() - INNER_tic
            all_adaptation_time.append(INNER_toc)

            acc, prec, rec, f1 = models.utils.calculate_metrics(task_predictions, task_labels)

            print('Support set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                        'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(support_loss), acc, prec, rec, f1))

            # Query set is now here!
            query_labels = torch.tensor(query_labels).to(device)
            query_input_dict = learner.rln.encode_text(query_text)
            with torch.no_grad():
                query_repr = learner.rln(query_input_dict)
                query_output = fpln(query_repr) # Output has size of torch.Size([16, 33]) [BATCH, CLASSES]
                query_loss = learner.loss_fn(query_output, query_labels)
            query_loss = query_loss.item()
            # print(output.detach().size())
            # output.detach().max(-1) max on each Batch, which will return [0] max, [1] indices
            query_output_softmax = F.softmax(query_output, -1)
            query_label_conf = query_output_softmax[np.arange(len(query_output_softmax)), query_labels] # Select labels in the softmax of 33 classes

            query_pred = models.utils.make_prediction(query_output.detach())
            query_acc, query_prec, query_rec, query_f1 = models.utils.calculate_metrics(query_pred.tolist(), query_labels.tolist())
            
            print('Query set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(query_loss), query_acc, query_prec, query_rec, query_f1))

            all_losses.append(query_loss)
            all_predictions.extend(query_pred.tolist())
            all_labels.extend(query_labels.tolist())
            all_label_conf.extend(query_label_conf.tolist())

    acc, prec, rec, f1 = models.utils.calculate_metrics(all_predictions, all_labels)
    print('Test metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
                'F1 score = {:.4f}'.format(np.mean(all_losses), acc, prec, rec, f1))
    return acc, prec, rec, f1, all_predictions, all_labels, all_label_conf, all_adaptation_time

In [10]:
tic = time.time()
print('----------Testing on test set starts here----------')

accuracies, precisions, recalls, f1s = [], [], [], []
all_adapt_time = []
# Data for Visualization: [data_idx, label, label_conf, pred]
data_for_visual = []

for test_dataset in test_datasets:
    print('Testing on {}'.format(test_dataset.__class__.__name__))
    test_dataloader = data.DataLoader(test_dataset, batch_size=mini_batch_size, shuffle=False,
                                      collate_fn=datasets.utils.batch_encode)
    acc, prec, rec, f1, all_pred, all_label, all_label_conf, all_adaptation_time = evaluate(dataloader=test_dataloader, updates=updates, 
                                                mini_batch_size=mini_batch_size, dataname=test_dataset.__class__.__name__)
    
    data_ids = [test_dataset.__class__.__name__ + str(i) for i in range(len(all_label))]
    data_for_visual.extend(list(zip(data_ids, all_label, all_label_conf, all_pred)))
    all_adapt_time.extend(all_adaptation_time)
#     print(data_ids)
#     print(all_label)
#     raise Exception("BREAKPOINT")
    
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)


print()
print("COPY PASTA - not really but ok")
for row in accuracies:
    print(row)
print()
print('Overall test metrics: Accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
            'F1 score = {:.4f}'.format(np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1s)))

toc = time.time() - tic
print(f"Total Time used: {toc//60} minutes")

----------Testing on test set starts here----------
Testing on YelpDataset
Query ID 0/475
Support set metrics: Loss = 0.3358, accuracy = 0.8875, precision = 0.8891, recall = 0.8800, F1 score = 0.8828
Query set metrics: Loss = 0.4699, accuracy = 0.8750, precision = 0.9167, recall = 0.8667, F1 score = 0.8732
Query ID 1/475
Support set metrics: Loss = 0.2935, accuracy = 0.9500, precision = 0.9527, recall = 0.9467, F1 score = 0.9481
Query set metrics: Loss = 0.9740, accuracy = 0.7500, precision = 0.6933, recall = 0.7933, F1 score = 0.7133
Query ID 2/475
Support set metrics: Loss = 0.2436, accuracy = 0.9875, precision = 0.9905, recall = 0.9867, F1 score = 0.9882
Query set metrics: Loss = 0.7234, accuracy = 0.6875, precision = 0.5600, recall = 0.7000, F1 score = 0.6111
Query ID 3/475
Support set metrics: Loss = 0.2578, accuracy = 0.9750, precision = 0.9771, recall = 0.9733, F1 score = 0.9749
Query set metrics: Loss = 1.4056, accuracy = 0.7500, precision = 0.7933, recall = 0.7667, F1 score = 

Support set metrics: Loss = 0.3366, accuracy = 0.9250, precision = 0.9255, recall = 0.9200, F1 score = 0.9215
Query set metrics: Loss = 0.7620, accuracy = 0.7500, precision = 0.7417, recall = 0.7625, F1 score = 0.7333
Query ID 36/475
Support set metrics: Loss = 0.3167, accuracy = 0.9125, precision = 0.9133, recall = 0.9067, F1 score = 0.9080
Query set metrics: Loss = 0.7091, accuracy = 0.6875, precision = 0.6667, recall = 0.7000, F1 score = 0.6285
Query ID 37/475
Support set metrics: Loss = 0.2961, accuracy = 0.9625, precision = 0.9655, recall = 0.9600, F1 score = 0.9615
Query set metrics: Loss = 0.9414, accuracy = 0.5000, precision = 0.4700, recall = 0.5367, F1 score = 0.4738
Query ID 38/475
Support set metrics: Loss = 0.2551, accuracy = 0.9750, precision = 0.9742, recall = 0.9733, F1 score = 0.9733
Query set metrics: Loss = 0.8741, accuracy = 0.6250, precision = 0.4833, recall = 0.6100, F1 score = 0.5355
Query ID 39/475
Support set metrics: Loss = 0.2823, accuracy = 0.9375, precision

Support set metrics: Loss = 0.3153, accuracy = 0.9000, precision = 0.9044, recall = 0.8933, F1 score = 0.8973
Query set metrics: Loss = 0.7483, accuracy = 0.6250, precision = 0.5133, recall = 0.5333, F1 score = 0.5111
Query ID 72/475
Support set metrics: Loss = 0.2837, accuracy = 0.9375, precision = 0.9440, recall = 0.9333, F1 score = 0.9361
Query set metrics: Loss = 1.0557, accuracy = 0.6250, precision = 0.6333, recall = 0.7300, F1 score = 0.6612
Query ID 73/475
Support set metrics: Loss = 0.3338, accuracy = 0.9125, precision = 0.9126, recall = 0.9067, F1 score = 0.9080
Query set metrics: Loss = 1.1300, accuracy = 0.6875, precision = 0.7000, recall = 0.7800, F1 score = 0.6905
Query ID 74/475
Support set metrics: Loss = 0.3188, accuracy = 0.9250, precision = 0.9316, recall = 0.9200, F1 score = 0.9227
Query set metrics: Loss = 1.0292, accuracy = 0.5625, precision = 0.6200, recall = 0.6667, F1 score = 0.6189
Query ID 75/475
Support set metrics: Loss = 0.3214, accuracy = 0.9000, precision

Support set metrics: Loss = 0.2777, accuracy = 0.9750, precision = 0.9742, recall = 0.9733, F1 score = 0.9733
Query set metrics: Loss = 1.0313, accuracy = 0.5000, precision = 0.5933, recall = 0.6000, F1 score = 0.5689
Query ID 107/475
Support set metrics: Loss = 0.2538, accuracy = 0.9625, precision = 0.9622, recall = 0.9600, F1 score = 0.9599
Query set metrics: Loss = 0.8828, accuracy = 0.7500, precision = 0.8033, recall = 0.7667, F1 score = 0.7433
Query ID 108/475
Support set metrics: Loss = 0.2890, accuracy = 0.9375, precision = 0.9379, recall = 0.9333, F1 score = 0.9348
Query set metrics: Loss = 0.7571, accuracy = 0.6875, precision = 0.5767, recall = 0.5933, F1 score = 0.5790
Query ID 109/475
Support set metrics: Loss = 0.2925, accuracy = 0.9500, precision = 0.9513, recall = 0.9467, F1 score = 0.9464
Query set metrics: Loss = 1.6752, accuracy = 0.4375, precision = 0.4133, recall = 0.4800, F1 score = 0.4378
Query ID 110/475
Support set metrics: Loss = 0.3048, accuracy = 0.9500, preci

Support set metrics: Loss = 0.2611, accuracy = 0.9750, precision = 0.9780, recall = 0.9733, F1 score = 0.9749
Query set metrics: Loss = 0.6198, accuracy = 0.6875, precision = 0.7333, recall = 0.6500, F1 score = 0.6643
Query ID 142/475
Support set metrics: Loss = 0.2383, accuracy = 0.9875, precision = 0.9875, recall = 0.9867, F1 score = 0.9867
Query set metrics: Loss = 0.6374, accuracy = 0.7500, precision = 0.6476, recall = 0.7000, F1 score = 0.6597
Query ID 143/475
Support set metrics: Loss = 0.2935, accuracy = 0.9500, precision = 0.9560, recall = 0.9467, F1 score = 0.9495
Query set metrics: Loss = 0.4345, accuracy = 0.7500, precision = 0.8133, recall = 0.8200, F1 score = 0.7576
Query ID 144/475
Support set metrics: Loss = 0.2788, accuracy = 0.9375, precision = 0.9481, recall = 0.9333, F1 score = 0.9371
Query set metrics: Loss = 1.7160, accuracy = 0.4375, precision = 0.2800, recall = 0.2833, F1 score = 0.2788
Query ID 145/475
Support set metrics: Loss = 0.2465, accuracy = 0.9875, preci

...

Support set metrics: Loss = 0.4834, accuracy = 0.9375, precision = 0.9500, recall = 0.9300, F1 score = 0.9314
Query set metrics: Loss = 0.4853, accuracy = 0.8750, precision = 0.9062, recall = 0.9062, F1 score = 0.8810
Query ID 313/475
Support set metrics: Loss = 0.4810, accuracy = 0.9000, precision = 0.9137, recall = 0.8900, F1 score = 0.8784
Query set metrics: Loss = 0.4579, accuracy = 0.8750, precision = 0.8958, recall = 0.9333, F1 score = 0.9028
Query ID 314/475
Support set metrics: Loss = 0.5070, accuracy = 0.9125, precision = 0.9164, recall = 0.9100, F1 score = 0.9061
Query set metrics: Loss = 0.4325, accuracy = 0.9375, precision = 0.8214, recall = 0.8571, F1 score = 0.8367
Query ID 315/475
Support set metrics: Loss = 0.5306, accuracy = 0.8625, precision = 0.8847, recall = 0.8100, F1 score = 0.8031
Query set metrics: Loss = 0.7582, accuracy = 0.8125, precision = 0.9286, recall = 0.8333, F1 score = 0.8680
Query ID 316/475
Support set metrics: Loss = 0.4647, accuracy = 0.9250, preci

Support set metrics: Loss = 0.5302, accuracy = 0.9125, precision = 0.8990, recall = 0.9100, F1 score = 0.8991
Query set metrics: Loss = 0.5444, accuracy = 0.8125, precision = 0.8125, recall = 0.7500, F1 score = 0.7708
Query ID 348/475
Support set metrics: Loss = 0.5081, accuracy = 0.9250, precision = 0.9367, recall = 0.9100, F1 score = 0.9106
Query set metrics: Loss = 0.8178, accuracy = 0.6875, precision = 0.5833, recall = 0.6111, F1 score = 0.5630
Query ID 349/475
Support set metrics: Loss = 0.5318, accuracy = 0.9000, precision = 0.9045, recall = 0.9000, F1 score = 0.8940
Query set metrics: Loss = 0.7549, accuracy = 0.8125, precision = 0.8571, recall = 0.8929, F1 score = 0.8190
Query ID 350/475
Support set metrics: Loss = 0.4466, accuracy = 0.9375, precision = 0.9385, recall = 0.9300, F1 score = 0.9268
Query set metrics: Loss = 0.5191, accuracy = 0.8125, precision = 0.8333, recall = 0.7708, F1 score = 0.7833
Query ID 351/475
Support set metrics: Loss = 0.4873, accuracy = 0.8500, preci

Support set metrics: Loss = 0.4621, accuracy = 0.9250, precision = 0.9182, recall = 0.9200, F1 score = 0.9147
Query set metrics: Loss = 0.8063, accuracy = 0.8125, precision = 0.7619, recall = 0.7143, F1 score = 0.6905
Query ID 383/475
Support set metrics: Loss = 0.5726, accuracy = 0.8750, precision = 0.8859, recall = 0.8700, F1 score = 0.8618
Query set metrics: Loss = 0.6455, accuracy = 0.8750, precision = 0.9286, recall = 0.9238, F1 score = 0.9079
Query ID 384/475
Support set metrics: Loss = 0.5179, accuracy = 0.8625, precision = 0.8662, recall = 0.8400, F1 score = 0.8303
Query set metrics: Loss = 1.2066, accuracy = 0.6250, precision = 0.5833, recall = 0.6500, F1 score = 0.5633
Query ID 385/475
Support set metrics: Loss = 0.5355, accuracy = 0.9500, precision = 0.9514, recall = 0.9500, F1 score = 0.9470
Query set metrics: Loss = 0.8462, accuracy = 0.6875, precision = 0.6562, recall = 0.6771, F1 score = 0.6476
Query ID 386/475
Support set metrics: Loss = 0.4511, accuracy = 0.9125, preci

Support set metrics: Loss = 0.4864, accuracy = 0.9250, precision = 0.9292, recall = 0.9100, F1 score = 0.8982
Query set metrics: Loss = 0.7024, accuracy = 0.7500, precision = 0.8000, recall = 0.8000, F1 score = 0.7667
Query ID 418/475
Support set metrics: Loss = 0.5026, accuracy = 0.9250, precision = 0.9269, recall = 0.9200, F1 score = 0.9183
Query set metrics: Loss = 0.7735, accuracy = 0.7500, precision = 0.6667, recall = 0.6458, F1 score = 0.6208
Query ID 419/475
Support set metrics: Loss = 0.5522, accuracy = 0.8250, precision = 0.8335, recall = 0.8400, F1 score = 0.8252
Query set metrics: Loss = 0.8886, accuracy = 0.5000, precision = 0.7222, recall = 0.4583, F1 score = 0.5222
Query ID 420/475
Support set metrics: Loss = 0.5353, accuracy = 0.8875, precision = 0.8963, recall = 0.9000, F1 score = 0.8906
Query set metrics: Loss = 1.1614, accuracy = 0.6250, precision = 0.4074, recall = 0.5556, F1 score = 0.4593
Query ID 421/475
Support set metrics: Loss = 0.5236, accuracy = 0.8625, preci

Support set metrics: Loss = 0.4908, accuracy = 0.9125, precision = 0.9181, recall = 0.9100, F1 score = 0.9066
Query set metrics: Loss = 0.1905, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 453/475
Support set metrics: Loss = 0.4830, accuracy = 0.9375, precision = 0.9409, recall = 0.9300, F1 score = 0.9314
Query set metrics: Loss = 0.4850, accuracy = 0.8125, precision = 0.8958, recall = 0.8646, F1 score = 0.8571
Query ID 454/475
Support set metrics: Loss = 0.5271, accuracy = 0.9125, precision = 0.9277, recall = 0.9100, F1 score = 0.9107
Query set metrics: Loss = 0.4223, accuracy = 0.8750, precision = 0.8000, recall = 0.8500, F1 score = 0.8167
Query ID 455/475
Support set metrics: Loss = 0.5046, accuracy = 0.8875, precision = 0.8903, recall = 0.9000, F1 score = 0.8862
Query set metrics: Loss = 0.4743, accuracy = 0.8125, precision = 0.7593, recall = 0.8148, F1 score = 0.7704
Query ID 456/475
Support set metrics: Loss = 0.4991, accuracy = 0.9000, preci

In [11]:
_model_path0 = os.path.splitext(model_path)[0]
csv_filename = _model_path0 + "_update"+ str(updates) +"_results.csv" # for selective replay
with open(csv_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["data_idx", "label", "label_conf", "pred"])
    csv_writer.writerows(data_for_visual)
print(f"Done writing CSV File at {csv_filename}")

Done writing CSV File at /data/model_runs/original_oml/aOML-order1-2023-03-15-no12/OML-order1-id4-2023-03-15_14-27-32.837449_update5_results.csv


In [12]:
# Log Time for Inference
_model_path0 = os.path.splitext(model_path)[0]
time_txt_filename = _model_path0 + "_update"+ str(updates) +"_time_inference.csv" 
with open(time_txt_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["time_id", "time"])
    csv_writer.writerow(["Total Time", f"{toc//60} minutes"])
    csv_writer.writerow(["mean Adapt Time", f"{np.mean(all_adapt_time)} s"])
print(f"Done writing Time CSV File at {time_txt_filename}")

Done writing Time CSV File at /data/model_runs/original_oml/aOML-order1-2023-03-15-no12/OML-order1-id4-2023-03-15_14-27-32.837449_update5_time_inference.csv
